In [2]:
from transformers import GPTNeoXForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/pythia-70m")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model

In [1]:
import torch

linear = torch.nn.Linear(768, 768)

In [7]:
lin_weight = linear.weight

In [14]:
u, s, v = torch.svd(lin_weight)
u.shape, s.shape, v.shape

(torch.Size([768, 768]), torch.Size([768]), torch.Size([768, 768]))

In [15]:
torch.dist(lin_weight, torch.matmul(torch.matmul(u, torch.diag_embed(s)), v.mT))

tensor(2.8584e-05, grad_fn=<DistBackward0>)

In [17]:
u, s, v = torch.svd_lowrank(lin_weight)
u.shape, s.shape, v.shape

(torch.Size([768, 6]), torch.Size([6]), torch.Size([768, 6]))

In [18]:
torch.dist(lin_weight, torch.matmul(torch.matmul(u, torch.diag_embed(s)), v.mT))

tensor(15.7942, grad_fn=<DistBackward0>)

In [19]:
768 * 256 * 2, 768 * 768

(393216, 589824)

In [6]:
1024 * 8 + 3072 * 8, 1024 * 4096, 1024 * 1024 * 3

(32768, 4194304, 3145728)

In [1]:
768 * 768 * 3, 768 * 8 * 2 * 3

(1769472, 36864)

In [7]:
1024 * 1024

1048576

In [8]:
1024 * 8 + 8 * 1024

16384

In [1]:
1_997_624_552 - 2_775_208_960

-777584408